In [1]:
!nvidia-smi

Wed Oct  6 21:17:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:41:00.0 Off |                    0 |
| N/A   18C    P0    49W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5, MonoBERT
from trectools import TrecRun
import ir_datasets
monoT5Reranker = MonoT5()
import pandas as pd
monoBert = MonoBERT()


DIR='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/'
DIR_v2='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-marco-v2-ecir22/'

2021-10-06 21:19:57 [INFO] loader: Loading faiss with AVX2 support.
2021-10-06 21:19:57 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
2021-10-06 21:19:57 [INFO] loader: Loading faiss.
2021-10-06 21:19:57 [INFO] loader: Successfully loaded faiss.


In [3]:
def load_topics(version, file):
    import pandas as pd
    return pd.read_csv('../../Data/navigational-topics-and-qrels-ms-marco-v' + str(version) + '/' + file, sep='\t', names=['num', 'query'])

df_popular_queries = load_topics(1, 'topics.msmarco-entrypage-popular.tsv')
df_random_queries = load_topics(1, 'topics.msmarco-entrypage-random.tsv')
df_popular_run = TrecRun(DIR + 'entrypage-popular/run.ms-marco-content.bm25-default.txt')
df_random_run = TrecRun(DIR + 'entrypage-random/run.ms-marco-content.bm25-default.txt')

df_dl_queries = pd.read_csv('../../Data/dl19+20/dl19+20-queries.tsv', sep='\t', names=['num', 'query'])
df_dl_run = TrecRun(DIR + 'dl19+20/run.ms-marco-content.bm25-default.txt')

df_popular_queries_v2 = load_topics(2, 'topics.msmarco-v2-entrypage-popular.tsv')
df_random_queries_v2 = load_topics(2, 'topics.msmarco-v2-entrypage-random.tsv')
df_popular_run_v2 = TrecRun(DIR_v2 + 'entrypage-popular/run.msmarco-doc-v2.bm25-default.txt')
df_random_run_v2 = TrecRun(DIR_v2 + 'entrypage-random/run.msmarco-doc-v2.bm25-default.txt')

In [13]:
ALL_DOC_IDS = []
for i in [df_popular_run, df_random_run, df_popular_run_v2, df_random_run_v2, df_dl_run]:
    ALL_DOC_IDS += [j for j in i.run_data.docid.unique()]

ALL_DOC_IDS = set(ALL_DOC_IDS)

In [37]:
def parse_passages(i):
    import json
    i = json.loads(i)
    passages = []
    for t in range(1, len(i['passages'])):
        if t == 1:
            passages += [i['passages'][0][1] + i['passages'][1][1]]
        else:
            passages += [i['passages'][t][1]]
    
    return (i['id'], passages)

def load_documents(file_name):
    from tqdm import tqdm
    ret = {}
    with open('/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/' + file_name) as f:
        for i in tqdm(f):
            doc_id, passages = parse_passages(i)
            assert doc_id not in ret
            if doc_id in ALL_DOC_IDS:
                ret[doc_id] = passages
    return ret

DOC_ID_TO_PASSAGES = {}
DOC_ID_TO_PASSAGES.update(load_documents('msmarco-document-passages.jsonl'))
DOC_ID_TO_PASSAGES.update(load_documents('msmarco-document-passages-v2.jsonl'))

3213835it [02:45, 19407.86it/s]
11959635it [13:14, 15059.73it/s]


In [38]:
len(DOC_ID_TO_PASSAGES)

335619

In [41]:
def get_query_or_fail(df_queries, topic_number):
    ret = df_queries[df_queries['num'] == int(topic_number)]
    if len(ret) != 1:
        raise ValueError('Could not handle ' + str(topic_number))
    
    return ret.iloc[0]['query']

def docs_for_topic(df_run, topic_number):
    return df_run.run_data[df_run.run_data['query'] == int(topic_number)].docid

def rerank_with_model(topic, df_queries, df_run, model):
    query = get_query_or_fail(df_queries, topic)
    documents = []
    for i in docs_for_topic(df_run, topic)[:100]:
        for passage in DOC_ID_TO_PASSAGES[i]:
            documents += [Text(passage, {'docid': i}, 0)]
    ret = sorted(model.rerank(Query(query), documents), key=lambda i: i.score, reverse=True)

    returned_ids = set()
    for i in ret:
        if i.metadata['docid'] not in returned_ids:
            returned_ids.add(i.metadata['docid'])
            yield {'score': i.score, 'id': i.metadata['docid']}

def rerank(file_name, df_run, df_queries, model, tag):
    from tqdm import tqdm
    
    with open(file_name, 'w') as out_file:
        for topic in tqdm(df_queries.num):
            for i in zip(range(100), rerank_with_model(topic, df_queries, df_run, model)):
                out_file.write(str(topic) + ' Q0 ' + i[1]['id'] + ' ' + str(i[0] + 1) + ' ' + str(i[1]['score']) + ' ' + tag + '\n')

# Marco V1

In [42]:
rerank(DIR + 'entrypage-random/run.ms-marco-content.bm25-mono-t5-maxp.txt', df_random_run, df_random_queries.copy(), monoT5Reranker, 'mono-t5-maxp-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:07<11:56,  7.31s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:08<06:26,  3.98s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

 22%|██▏       | 22/99 [03:13<10:03,  7.83s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 23%|██▎       | 23/99 [03:24<10:53,  8.60s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 24%|██▍       | 24/99 [03:29<09:30,  7.60s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 44%|████▍     | 44/99 [05:42<06:40,  7.28s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 45%|████▌     | 45/99 [05:51<06:48,  7.56s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 46%|████▋     | 46/99 [05:59<06:45,  7.65s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 68%|██████▊   | 67/99 [09:18<04:48,  9.03s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 69%|██████▊   | 68/99 [09:28<04:46,  9.23s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 70%|██████▉   | 69/99 [09:35<04:15,  8.51s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 92%|█████████▏| 91/99 [12:22<01:20, 10.00s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 93%|█████████▎| 92/99 [12:29<01:03,  9.01s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 94%|█████████▍| 93/99 [12:35<00:49,  8.25s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

In [43]:
rerank(DIR +  'entrypage-popular/run.ms-marco-content.bm25-mono-t5-maxp.txt', df_popular_run, df_popular_queries.copy(), monoT5Reranker, 'mono-t5-maxp-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:04<08:03,  4.94s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:09<07:39,  4.74s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

 22%|██▏       | 22/99 [02:29<12:16,  9.57s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 23%|██▎       | 23/99 [02:33<09:54,  7.82s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 24%|██▍       | 24/99 [02:43<10:38,  8.51s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 44%|████▍     | 44/99 [04:40<05:55,  6.46s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 45%|████▌     | 45/99 [04:48<06:08,  6.83s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 46%|████▋     | 46/99 [04:53<05:29,  6.23s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 67%|██████▋   | 66/99 [07:32<05:42, 10.37s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 68%|██████▊   | 67/99 [07:40<05:17,  9.93s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 69%|██████▊   | 68/99 [07:46<04:31,  8.74s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 90%|████████▉ | 89/99 [10:35<01:04,  6.47s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 91%|█████████ | 90/99 [10:41<00:56,  6.23s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 92%|█████████▏| 91/99 [10:46<00:48,  6.02s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

# Marco V2

In [44]:
rerank(DIR_v2 + 'entrypage-random/run.ms-marco-content.bm25-mono-t5-maxp.txt', df_random_run_v2, df_random_queries_v2.copy(), monoT5Reranker, 'mono-t5-maxp-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:05<09:01,  5.53s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:10<08:05,  5.01s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

 22%|██▏       | 22/99 [03:08<10:01,  7.81s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 23%|██▎       | 23/99 [03:20<11:24,  9.01s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 24%|██▍       | 24/99 [03:30<11:31,  9.23s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 44%|████▍     | 44/99 [06:22<10:12, 11.14s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 45%|████▌     | 45/99 [06:32<09:44, 10.82s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 46%|████▋     | 46/99 [06:43<09:45, 11.04s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 67%|██████▋   | 66/99 [09:37<03:47,  6.90s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 68%|██████▊   | 67/99 [09:55<05:26, 10.21s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 69%|██████▊   | 68/99 [10:05<05:05,  9.87s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 89%|████████▉ | 88/99 [12:32<01:22,  7.46s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 90%|████████▉ | 89/99 [12:44<01:27,  8.79s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 91%|█████████ | 90/99 [12:50<01:12,  8.04s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

In [45]:
rerank(DIR_v2 +  'entrypage-popular/run.ms-marco-content.bm25-mono-t5-maxp.txt', df_popular_run_v2, df_popular_queries_v2.copy(), monoT5Reranker, 'mono-t5-maxp-at-bm25')

  0%|          | 0/99 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  1%|          | 1/99 [00:09<16:06,  9.86s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  2%|▏         | 2/99 [00:19<15:43,  9.73s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

 22%|██▏       | 22/99 [03:30<14:05, 10.97s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 23%|██▎       | 23/99 [03:36<12:18,  9.72s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 24%|██▍       | 24/99 [03:50<13:26, 10.75s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 44%|████▍     | 44/99 [06:26<06:59,  7.63s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 45%|████▌     | 45/99 [06:35<07:21,  8.17s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 46%|████▋     | 46/99 [06:40<06:20,  7.17s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 67%|██████▋   | 66/99 [09:57<06:00, 10.92s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 68%|██████▊   | 67/99 [10:12<06:29, 12.19s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 69%|██████▊   | 68/99 [10:18<05:15, 10.19s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

 90%|████████▉ | 89/99 [14:01<01:50, 11.01s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 91%|█████████ | 90/99 [14:16<01:48, 12.06s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 92%|█████████▏| 91/99 [14:22<01:23, 10.46s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to dup

# Rerank with MonoBERT

In [46]:
rerank(DIR + 'entrypage-random/run.ms-marco-content.bm25-mono-bert-maxp.txt', df_random_run, df_random_queries.copy(), monoBert, 'mono-bert-maxp-at-bm25')

100%|██████████| 99/99 [41:06<00:00, 24.91s/it]


In [47]:
rerank(DIR +  'entrypage-popular/run.ms-marco-content.bm25-mono-bert-maxp.txt', df_popular_run, df_popular_queries.copy(), monoBert, 'mono-bert-maxp-at-bm25')

100%|██████████| 99/99 [36:40<00:00, 22.23s/it]


In [48]:
rerank(DIR_v2 + 'entrypage-random/run.ms-marco-content.bm25-mono-bert-maxp.txt', df_random_run_v2, df_random_queries_v2.copy(), monoBert, 'mono-bert-maxp-at-bm25')

100%|██████████| 99/99 [43:41<00:00, 26.48s/it]


In [49]:
rerank(DIR_v2 +  'entrypage-popular/run.ms-marco-content.bm25-mono-bert-maxp.txt', df_popular_run_v2, df_popular_queries_v2.copy(), monoBert, 'mono-bert-maxp-at-bm25')

100%|██████████| 99/99 [47:33<00:00, 28.82s/it]


# Rerank remaining topics for DL 19 + 20 

In [50]:
import pandas as pd

df_dl_queries = pd.read_csv('../../Data/dl19+20/dl19+20-queries.tsv', sep='\t', names=['num', 'query'])

df_dl_run = TrecRun(DIR + 'dl19+20/run.ms-marco-content.bm25-default.txt')

In [51]:
# delete
ALL_DOC_IDS = []
for i in [df_popular_run, df_random_run, df_popular_run_v2, df_random_run_v2, df_dl_run]:
    ALL_DOC_IDS += [j for j in i.run_data.docid.unique()]

ALL_DOC_IDS = set(ALL_DOC_IDS)

In [52]:
# delete
def parse_passages(i):
    import json
    i = json.loads(i)
    passages = []
    for t in range(1, len(i['passages'])):
        if t == 1:
            passages += [i['passages'][0][1] + i['passages'][1][1]]
        else:
            passages += [i['passages'][t][1]]
    
    return (i['id'], passages)

def load_documents(file_name):
    from tqdm import tqdm
    ret = {}
    with open('/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/' + file_name) as f:
        for i in tqdm(f):
            doc_id, passages = parse_passages(i)
            assert doc_id not in ret
            if doc_id in ALL_DOC_IDS:
                ret[doc_id] = passages
    return ret

DOC_ID_TO_PASSAGES = {}
DOC_ID_TO_PASSAGES.update(load_documents('msmarco-document-passages.jsonl'))
DOC_ID_TO_PASSAGES.update(load_documents('msmarco-document-passages-v2.jsonl'))

3213835it [02:51, 18696.03it/s]
11959635it [13:34, 14682.01it/s]


In [53]:
rerank(DIR + 'dl19+20/run.ms-marco-content.bm25-mono-t5-maxp.txt', df_dl_run, df_dl_queries.copy(), monoT5Reranker, 'mono-t5-maxp-at-bm25')

  0%|          | 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  0%|          | 1/400 [00:08<53:35,  8.06s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  0%|          | 2/400 [00:22<1:19:42, 12.02s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicate

  5%|▌         | 21/400 [03:51<53:39,  8.50s/it]  /usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  6%|▌         | 22/400 [03:57<50:32,  8.02s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
  6%|▌         | 23/400 [04:04<46:50,  7.46s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead t

 10%|█         | 42/400 [07:51<1:13:56, 12.39s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 11%|█         | 43/400 [08:03<1:14:05, 12.45s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 11%|█         | 44/400 [08:09<1:02:36, 10.55s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may le

 16%|█▌        | 63/400 [11:09<53:06,  9.45s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 16%|█▌        | 64/400 [11:19<55:03,  9.83s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 16%|█▋        | 65/400 [11:25<47:47,  8.56s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to 

 21%|██        | 84/400 [14:07<37:04,  7.04s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 21%|██▏       | 85/400 [14:21<47:40,  9.08s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 22%|██▏       | 86/400 [14:28<43:51,  8.38s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to 

...

 79%|███████▉  | 315/400 [53:58<14:50, 10.48s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 79%|███████▉  | 316/400 [54:11<15:27, 11.05s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 79%|███████▉  | 317/400 [54:26<17:08, 12.39s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead 

 84%|████████▍ | 336/400 [57:10<11:04, 10.38s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 84%|████████▍ | 337/400 [57:17<09:53,  9.42s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 84%|████████▍ | 338/400 [57:24<08:55,  8.64s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead 

 89%|████████▉ | 357/400 [1:00:20<06:21,  8.86s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 90%|████████▉ | 358/400 [1:00:26<05:36,  8.00s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 90%|████████▉ | 359/400 [1:00:36<05:57,  8.72s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may

 94%|█████████▍| 378/400 [1:03:11<03:13,  8.81s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 95%|█████████▍| 379/400 [1:03:19<03:03,  8.76s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
 95%|█████████▌| 380/400 [1:03:29<02:58,  8.90s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may

100%|█████████▉| 399/400 [1:06:56<00:10, 10.13s/it]/usr/local/lib/python3.6/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 400/400 [1:07:07<00:00, 10.07s/it]


In [54]:
rerank(DIR + 'dl19+20/run.ms-marco-content.bm25-mono-bert-maxp.txt', df_dl_run, df_dl_queries.copy(), monoBert, 'mono-bert-maxp-at-bm25')

100%|██████████| 400/400 [3:23:04<00:00, 30.46s/it]  
